# Data Cleansing


## Imports

In [42]:
import psycopg2
import db_config as creds
import os
import time
from stat import * # ST_SIZE etc

## Database functions

In [43]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)
    
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

def disconnect(conn,cursor):
    
    conn.close()

In [44]:
# Connecting to DB
#conn, cursor = connect()

#insert_image = """
#    INSERT INTO images (filename) VALUES('UA502')       
#    """

# Execute SQL Command and commit to DB
#cursor.execute(insert_image)
#conn.commit()

# Disconnect from DB
#disconnect(conn, cursor)

In [45]:
for subdir, dirs, files in os.walk(r'C:\DS Project\Images\Mike\Original'):
    for filename in files:
        filepath = subdir + os.sep + filename

        if filepath.endswith(".JPG") or filepath.endswith(".png"):
            stat = os.stat(filepath)
            print (filepath, time.asctime(time.localtime(stat[ST_MTIME])))
          

C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0005.JPG Wed Mar 11 07:10:52 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0006.JPG Wed Mar 11 07:10:54 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0007.JPG Wed Mar 11 07:10:56 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0008.JPG Wed Mar 11 07:10:58 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0009.JPG Wed Mar 11 07:11:02 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0010.JPG Wed Mar 11 07:11:02 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0011.JPG Wed Mar 11 07:11:02 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0012.JPG Wed Mar 11 07:11:02 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0013.JPG Wed Mar 11 07:11:04 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0014.JPG Wed Mar 11 07:11:04 2020
C:\DS Project\Images\Mike\Orig

C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0802.JPG Thu Mar 12 14:58:12 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0803.JPG Thu Mar 12 15:03:58 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0804.JPG Thu Mar 12 15:03:58 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0805.JPG Thu Mar 12 15:03:58 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0806.JPG Thu Mar 12 15:04:02 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0807.JPG Thu Mar 12 15:08:44 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0808.JPG Thu Mar 12 15:08:44 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0809.JPG Thu Mar 12 15:10:54 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0810.JPG Thu Mar 12 15:10:54 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0811.JPG Thu Mar 12 15:10:54 2020
C:\DS Project\Images\Mike\Orig

C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1563.JPG Sat Mar 14 09:32:52 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1564.JPG Sat Mar 14 09:32:52 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1565.JPG Sat Mar 14 09:32:54 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1566.JPG Sat Mar 14 09:33:06 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1567.JPG Sat Mar 14 09:33:06 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1568.JPG Sat Mar 14 09:33:06 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1569.JPG Sat Mar 14 09:35:28 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1570.JPG Sat Mar 14 09:35:28 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1571.JPG Sat Mar 14 09:35:28 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk1572.JPG Sat Mar 14 09:37:12 2020
C:\DS Project\Images\Mike\Orig

C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2376.JPG Fri Apr  3 10:23:16 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2377.JPG Fri Apr  3 10:23:54 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2378.JPG Fri Apr  3 10:24:32 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2379.JPG Fri Apr  3 10:24:32 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2380.JPG Fri Apr  3 10:26:18 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2381.JPG Fri Apr  3 10:26:18 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2382.JPG Fri Apr  3 10:26:18 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2383.JPG Fri Apr  3 10:26:20 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2384.JPG Fri Apr  3 10:26:48 2020
C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk2385.JPG Fri Apr  3 10:26:48 2020
C:\DS Project\Images\Mike\Orig

C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0464.JPG Sat Apr 11 10:36:34 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0465.JPG Sat Apr 11 10:36:34 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0466.JPG Sat Apr 11 10:39:22 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0467.JPG Sat Apr 11 10:42:18 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0468.JPG Sat Apr 11 10:42:18 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0469.JPG Sat Apr 11 10:42:20 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0470.JPG Sat Apr 11 10:43:08 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0471.JPG Sat Apr 11 10:47:30 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0472.JPG Sat Apr 11 10:49:26 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0473.JPG Sat Apr 11 10:49:28 2020
C:\DS Project\Images\Mike\Original\02 April 10 14 2020\02Mk0474.JPG Sa

...

C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3716.JPG Wed Jul 22 17:21:26 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3717.JPG Wed Jul 22 17:21:26 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3718.JPG Wed Jul 22 17:25:48 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3719.JPG Wed Jul 22 17:25:48 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3720.JPG Wed Jul 22 17:26:16 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3721.JPG Wed Jul 22 17:26:16 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3722.JPG Wed Jul 22 17:26:42 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3723.JPG Wed Jul 22 17:26:42 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3724.JPG Wed Jul 22 17:26:44 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3725.JPG Wed Jul 22 17:26:44 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk3726.JPG Wed Jul 22 17

C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4447.JPG Thu Jul 23 13:37:54 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4448.JPG Thu Jul 23 13:37:54 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4449.JPG Thu Jul 23 13:37:56 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4450.JPG Thu Jul 23 13:39:36 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4451.JPG Thu Jul 23 13:39:36 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4452.JPG Thu Jul 23 13:42:42 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4453.JPG Thu Jul 23 13:42:44 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4454.JPG Thu Jul 23 13:42:44 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4455.JPG Thu Jul 23 13:45:20 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4456.JPG Thu Jul 23 13:45:20 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4457.JPG Thu Jul 23 13

C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4955.JPG Fri Jul 24 08:30:28 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4956.JPG Fri Jul 24 08:30:32 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4957.JPG Fri Jul 24 08:30:32 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4958.JPG Fri Jul 24 08:30:32 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4959.JPG Fri Jul 24 08:31:26 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4960.JPG Fri Jul 24 08:31:26 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4961.JPG Fri Jul 24 08:39:46 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4962.JPG Fri Jul 24 08:39:46 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4963.JPG Fri Jul 24 08:39:46 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4964.JPG Fri Jul 24 08:50:18 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk4965.JPG Fri Jul 24 08

C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5587.JPG Fri Jul 24 17:11:16 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5588.JPG Fri Jul 24 17:15:52 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5589.JPG Fri Jul 24 17:15:52 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5590.JPG Fri Jul 24 17:16:36 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5591.JPG Fri Jul 24 17:18:28 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5592.JPG Fri Jul 24 17:18:30 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5593.JPG Fri Jul 24 17:18:32 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5594.JPG Fri Jul 24 17:19:46 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5595.JPG Fri Jul 24 17:19:46 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5596.JPG Fri Jul 24 17:20:44 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk5597.JPG Fri Jul 24 17

C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6100.JPG Sat Jul 25 17:55:22 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6101.JPG Sat Jul 25 17:55:52 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6102.JPG Sat Jul 25 18:00:52 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6103.JPG Sat Jul 25 18:00:52 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6104.JPG Sat Jul 25 18:00:54 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6105.JPG Sat Jul 25 18:07:12 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6106.JPG Sat Jul 25 18:07:14 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6107.JPG Sat Jul 25 18:12:30 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6108.JPG Sat Jul 25 18:12:30 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6109.JPG Sat Jul 25 18:12:32 2020
C:\DS Project\Images\Mike\Original\08 July 14 26 2020\08Mk6110.JPG Sat Jul 25 18

In [ ]:
import os, time
from stat import * # ST_SIZE etc

try:
    st = os.stat("C:\DS Project\Images\Mike\Original\01 Mar 11 April 04 2020\01Mk0005.JPG")
except IOError:
    print "failed to get information about", file
else:
    print "file size:", st[ST_SIZE]
    print "file modified:", time.asctime(time.localtime(st[ST_MTIME]))